In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import re
import zipfile
import io

In [2]:
def simple_get(url):

    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        print('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

def is_good_response(resp):
    
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)

In [3]:
def get_download_links_from_url(url):
    
    html=BeautifulSoup(simple_get(url),'html.parser')
    links= get_download_links_from_html(html)
    
    page=1
    
    while simple_get(url+'/partial/'+str(page)) is not None:
        html=BeautifulSoup(simple_get(url+'/partial/'+str(page)),'html.parser')
        links.extend(get_download_links_from_html(html))
        page+=1
    
    return links
 
def get_download_links_from_html(html):
    
    links=[]
    
    for link_text in html.select('a'):
        if extract_download_link(link_text) is not None:
            links.append(extract_download_link(link_text))
    
    return links

def extract_download_link(text):
    
    try:
        download_link= "https://code"+ re.search('"http://code(.*?)"',str(text)).group(1)
        download_link= re.sub("&amp;","&",download_link)
        return download_link
    
    except AttributeError:
        return None

In [4]:
def download_file_from(link,path):
    r = get(link,stream=True)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(path=path)

In [5]:
URL= 'https://www.go-hero.net/jam/'    
PATH="Solutions_in_python/Problem_"

dir_number=1
years= ['08','09','10','11','12','13','14','15','16','17']

In [6]:
for year in years:
    
    problem_number=1
    round_number=0
    url= URL+f'{year}/solutions/{round_number}/{problem_number}/Python'
    
    while simple_get(url) is not None:
        
        while simple_get(url) is not None:
            
            links= get_download_links_from_url(url)
            !mkdir {PATH}{dir_number}
            for link in links:
                download_file_from(link,PATH+str(dir_number))
            
            problem_number+=1
            dir_number+=1
            url= URL+f'{year}/solutions/{round_number}/{problem_number}/Python'
        
        problem_number=1
        round_number+=1
        url= URL+f'{year}/solutions/{round_number}/{problem_number}/Python'
    
    

IndexError: string index out of range